# Web analytics Phase 2

## STEP1: Data scraping 
The following code will scrap all the pitchers' stat from each MLB team from season 2003-2023

In [101]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_espn_stats(team, year):
    url = f'https://www.espn.com/mlb/team/stats/_/type/pitching/name/{team}/season/{year}/seasontype/2'
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
    
        soup = BeautifulSoup(response.content, 'html.parser')
        tables = soup.find_all('tbody', class_='Table__TBODY')
        
        if len(tables) < 2:  # Check if the expected tables are present
            print(f"Couldn't find the required tables for team {team} in year {year}.")
            return pd.DataFrame()

        names_table = tables[0]
        names_rows = names_table.find_all('tr')
        names = [row.find('td').get_text(strip=True) for row in names_rows]

        stats_table = tables[1]
        stats_rows = stats_table.find_all('tr')
        stats = [[td.get_text(strip=True) for td in row.find_all('td')] for row in stats_rows]
    else:
        print(f"Failed to retrieve data for team {team} in year {year}. Status code: {response.status_code}")
        return pd.DataFrame()  
    
    if names and stats:
       
        names = [[name, team.upper(), year] for name in names]  
        names_df = pd.DataFrame(names, columns=['Name', 'Team', 'Year'])
        stats_df = pd.DataFrame(stats)

        combined_df = pd.concat([names_df, stats_df], axis=1)

        column_names = [
            'Name', 'Team', 'Year', 'GP', 'GS', 'QS', 'W', 'L', 'SV', 'HLD', 'IP', 
            'H', 'ER', 'HR', 'BB', 'K', 'K/9', 'P/S', 'WAR', 'WHIP', 'ERA'
        ]
        combined_df.columns = column_names
        return combined_df
    else:
        return pd.DataFrame()  

teams = ['ari', 'atl', 'bal', 'bos', 'chc', 'chw', 'cin', 'cle', 'col', 'det', 'mia', 'hou', 'kc', 'laa','lad', 'nym', 'nyy', 'mil', 'min', 'oak', 'phi', 'pit', 'sd', 'sf', 'sea', 'stl', 'tb', 'tex', 'tor', 'wsh']
years = [str(year) for year in range(2023, 2002, -1)]

all_players_stats = pd.DataFrame()

for team in teams:
    for year in years:
        current_team_stats = scrape_espn_stats(team, year)
        all_players_stats = pd.concat([all_players_stats, current_team_stats], ignore_index=True)



print(all_players_stats)
all_players_stats.to_csv('all_players_stats.csv', index=False)

Couldn't find the required tables for team chw in year 2009.
Couldn't find the required tables for team chw in year 2008.
Couldn't find the required tables for team chw in year 2007.
Couldn't find the required tables for team chw in year 2006.
Couldn't find the required tables for team chw in year 2005.
Couldn't find the required tables for team chw in year 2004.
Couldn't find the required tables for team chw in year 2003.
Couldn't find the required tables for team mia in year 2011.
Couldn't find the required tables for team mia in year 2010.
Couldn't find the required tables for team mia in year 2009.
Couldn't find the required tables for team mia in year 2008.
Couldn't find the required tables for team mia in year 2007.
Couldn't find the required tables for team mia in year 2006.
Couldn't find the required tables for team mia in year 2005.
Couldn't find the required tables for team mia in year 2004.
Couldn't find the required tables for team mia in year 2003.
Couldn't find the requir

## STEP 2: Data cleaning

In [102]:
all_players_stats

,Name,Team,Year,GP,GS,QS,W,L,SV,HLD,...,H,ER,HR,BB,K,K/9,P/S,WAR,WHIP,ERA
0,Zac GallenSP,ARI,2023,34,34,20,17,9,0,0,...,188,81,22,47,220,9.4,95.5,4.4,1.12,3.47
1,Merrill KellySP,ARI,2023,30,30,18,12,8,0,0,...,143,65,20,69,187,9.5,93.4,3.9,1.19,3.29
2,Ryne NelsonSP,ARI,2023,29,27,7,8,8,0,0,...,159,85,24,46,96,6.0,84.5,0.0,1.42,5.31
3,Brandon PfaadtSP,ARI,2023,19,18,3,3,9,0,0,...,109,61,22,26,94,8.8,85.2,-0.4,1.41,5.72
4,Tommy HenrySP,ARI,2023,17,16,6,5,4,0,0,...,86,41,12,35,64,6.5,86.3,1.1,1.36,4.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16337,Matt ChicoSP,WSH,2010,1,1,0,0,0,0,0,...,6,2,0,0,3,5.4,79.0,0.1,1.20,3.60
16338,Joe BiseniusRP,WSH,2010,5,0,0,0,0,0,0,...,6,5,1,6,5,9.6,0.0,-0.2,2.57,9.64
16339,Garrett MockSP,WSH,2010,1,1,0,0,0,0,0,...,4,2,2,5,3,8.1,84.0,0.0,2.70,5.40
16340,Jason BergmannRP,WSH,2010,4,0,0,0,1,0,1,...,3,4,2,1,2,7.7,0.0,-0.2,1.71,15.43


In [103]:
#Delet row Where Name == "Total"
df = all_players_stats[all_players_stats['Name'] != 'Total']
df

,Name,Team,Year,GP,GS,QS,W,L,SV,HLD,...,H,ER,HR,BB,K,K/9,P/S,WAR,WHIP,ERA
0,Zac GallenSP,ARI,2023,34,34,20,17,9,0,0,...,188,81,22,47,220,9.4,95.5,4.4,1.12,3.47
1,Merrill KellySP,ARI,2023,30,30,18,12,8,0,0,...,143,65,20,69,187,9.5,93.4,3.9,1.19,3.29
2,Ryne NelsonSP,ARI,2023,29,27,7,8,8,0,0,...,159,85,24,46,96,6.0,84.5,0.0,1.42,5.31
3,Brandon PfaadtSP,ARI,2023,19,18,3,3,9,0,0,...,109,61,22,26,94,8.8,85.2,-0.4,1.41,5.72
4,Tommy HenrySP,ARI,2023,17,16,6,5,4,0,0,...,86,41,12,35,64,6.5,86.3,1.1,1.36,4.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16336,Jesse EnglishRP,WSH,2010,7,0,0,0,0,0,0,...,10,3,0,2,4,5.1,0.0,0.1,1.71,3.86
16337,Matt ChicoSP,WSH,2010,1,1,0,0,0,0,0,...,6,2,0,0,3,5.4,79.0,0.1,1.20,3.60
16338,Joe BiseniusRP,WSH,2010,5,0,0,0,0,0,0,...,6,5,1,6,5,9.6,0.0,-0.2,2.57,9.64
16339,Garrett MockSP,WSH,2010,1,1,0,0,0,0,0,...,4,2,2,5,3,8.1,84.0,0.0,2.70,5.40


In [104]:
# Remove asterisks from the 'Name' column
df['Name'] = df['Name'].str.replace(r'\*$', '', regex=True)

# Extract position abbreviations into a new 'Position' column
df['Position'] = df['Name'].str.extract(r'([A-Z]+)$')

# Remove position abbreviations from the 'Name' column
df['Name'] = df['Name'].str.replace(r'([A-Z]+)$', '').str.strip()

# Trim whitespace that may be left after removing the position
df['Name'] = df['Name'].str.strip()

# Check the result
print(df[['Name', 'Position']])

                 Name Position
0          Zac Gallen       SP
1       Merrill Kelly       SP
2         Ryne Nelson       SP
3      Brandon Pfaadt       SP
4         Tommy Henry       SP
...               ...      ...
16336   Jesse English       RP
16337      Matt Chico       SP
16338    Joe Bisenius       RP
16339    Garrett Mock       SP
16340  Jason Bergmann       RP

[15744 rows x 2 columns]


C:\Users\potat\AppData\Local\Temp\ipykernel_14964\2774336618.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name'] = df['Name'].str.replace(r'\*$', '', regex=True)
C:\Users\potat\AppData\Local\Temp\ipykernel_14964\2774336618.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Position'] = df['Name'].str.extract(r'([A-Z]+)$')
C:\Users\potat\AppData\Local\Temp\ipykernel_14964\2774336618.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Name'

In [105]:
df

,Name,Team,Year,GP,GS,QS,W,L,SV,HLD,...,ER,HR,BB,K,K/9,P/S,WAR,WHIP,ERA,Position
0,Zac Gallen,ARI,2023,34,34,20,17,9,0,0,...,81,22,47,220,9.4,95.5,4.4,1.12,3.47,SP
1,Merrill Kelly,ARI,2023,30,30,18,12,8,0,0,...,65,20,69,187,9.5,93.4,3.9,1.19,3.29,SP
2,Ryne Nelson,ARI,2023,29,27,7,8,8,0,0,...,85,24,46,96,6.0,84.5,0.0,1.42,5.31,SP
3,Brandon Pfaadt,ARI,2023,19,18,3,3,9,0,0,...,61,22,26,94,8.8,85.2,-0.4,1.41,5.72,SP
4,Tommy Henry,ARI,2023,17,16,6,5,4,0,0,...,41,12,35,64,6.5,86.3,1.1,1.36,4.15,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16336,Jesse English,WSH,2010,7,0,0,0,0,0,0,...,3,0,2,4,5.1,0.0,0.1,1.71,3.86,RP
16337,Matt Chico,WSH,2010,1,1,0,0,0,0,0,...,2,0,0,3,5.4,79.0,0.1,1.20,3.60,SP
16338,Joe Bisenius,WSH,2010,5,0,0,0,0,0,0,...,5,1,6,5,9.6,0.0,-0.2,2.57,9.64,RP
16339,Garrett Mock,WSH,2010,1,1,0,0,0,0,0,...,2,2,5,3,8.1,84.0,0.0,2.70,5.40,SP


In [106]:
df.to_csv('all_players_stats_cleaned.csv', index=False)

                    Name Team  Year  GP  GS  QS   W   L SV HLD  ...  ER  HR  \
383        Edwin Jackson  ARI  2010  21  21   8   6  10  0   0  ...  77  13   
609     Jackson Stephens  ATL  2023   5   0   0   0   0  0   0  ...   4   1   
625     Jackson Stephens  ATL  2022  39   1   0   3   3  2   2  ...  22   3   
643          Jay Jackson  ATL  2022   2   0   0   0   0  0   0  ...   0   0   
654         Luke Jackson  ATL  2021  71   0   0   2   2  0  31  ...  14   6   
...                  ...  ...   ...  ..  ..  ..  ..  .. ..  ..  ...  ..  ..   
15502      Edwin Jackson  TOR  2019   8   5   0   1   5  0   0  ...  35  12   
15983   Jackson Rutledge  WSH  2023   4   4   1   1   1  0   0  ...  15   4   
16011  Jackson Tetreault  WSH  2022   4   4   2   2   2  0   0  ...  12   4   
16157      Edwin Jackson  WSH  2017  13  13   7   5   6  0   0  ...  40  18   
16272      Edwin Jackson  WSH  2012  31  31  17  10  11  0   0  ...  85  23   

       BB    K  K/9    P/S   WAR  WHIP    ERA Posit